# Dataset

In [ ]:
# triplet predictions all methods file => Sentence Level Baseline(all entities are provided) and will be used to compare with Pair level ones(just two entities given per time)

In [ ]:
# Data loading and processing
import pandas as pd
import numpy as np
import json
whole_df = pd.read_csv("/content/Structured_whole_dataframe.csv")
whole_df = whole_df.drop("Unnamed: 0", axis = 1)
train_df = whole_df[whole_df["source_x"].astype(str).str.startswith("DDICorpus/Train/")]
train_df
train_df["type"] = train_df["type"].replace("No interaction", "no interaction")
train_df
test_df = whole_df[whole_df["source_x"].astype(str).str.startswith("DDICorpus/Test/")]
test_df
test_df["type"] = test_df["type"].replace("No interaction", "no interaction")
test_df
# example => train_df or test_df
train_df_records = train_df.to_dict(orient = "records")
train_df_records
test_df_records = test_df.to_dict(orient = "records")
test_df_records
# 1. Sample 10 examples of 'no interaction'
no_interaction_df = train_df[train_df['type'] == 'no interaction'].sample(n=10, random_state=42)

# 2. Sample 3 examples from each of the *other* types (mechanism, effect, etc.)
other_types_df = (
    train_df[train_df['type'] != 'no interaction']
    .groupby('type', group_keys=False)
    .sample(n=3, random_state=42)
)

# 3. Combine and shuffle
examples_df = pd.concat([no_interaction_df, other_types_df]).sample(frac=1, random_state=42).reset_index(drop=True)

examples_df["type"] = examples_df["type"].replace("No interaction", "no interaction")
examples_df
example_shots = examples_df.to_dict(orient="records")
example_shots
train_df_grouped = train_df.groupby("sentence_id")
train_df_grouped
group = train_df_grouped.get_group("DDI-DrugBank.d64.s87")
group
train_df["triplet"] = list(zip(train_df["e1_text"], train_df["type"], train_df["e2_text"]))
train_df
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = train_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))

    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]

    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("train_df_sentence_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)

with open("train_df_sentence_triplets.json","r") as f:
    train_triplets = json.load(f)

train_triplets[0]
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = test_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))

    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]

    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("test_df_sentence_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)
with open("test_df_sentence_triplets.json","r") as f:
    test_triplets = json.load(f)

test_triplets[0]
import json

# Group by unique sentence (or sentence_id if preferred)
grouped = train_df.groupby("sentence_text")

# Build JSON-like structure
sentence_entities = []

for sentence_text, group in grouped:
    # Collect unique entities from both drug1 and drug2
    entities = set(group["e1_text"]).union(set(group["e2_text"]))

    sentence_entities.append({
        "sentence": sentence_text,
        "entities": sorted(entities)  # optional: sort for consistency
    })

# Save to JSON
with open("train_df_sentence_entities.json", "w") as f:
    json.dump(sentence_entities, f, indent=2)

with open("train_df_sentence_entities.json","r") as f:
    train_entities = json.load(f)

train_entities[0]
train_triplets
few_shot_examples = train_triplets
few_shot_examples
import json

# Group by unique sentence (or sentence_id if preferred)
grouped = test_df.groupby("sentence_text")

# Build JSON-like structure
sentence_entities = []

for sentence_text, group in grouped:
    # Collect unique entities from both drug1 and drug2
    entities = set(group["e1_text"]).union(set(group["e2_text"]))

    sentence_entities.append({
        "sentence": sentence_text,
        "entities": sorted(entities)  # optional: sort for consistency
    })

# Save to JSON
with open("test_df_sentence_entities.json", "w") as f:
    json.dump(sentence_entities, f, indent=2)

with open("test_df_sentence_entities.json","r") as f:
    test_entities = json.load(f)

test_entities[0]
len(train_entities)
len(test_entities)
examples_df
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = examples_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))

    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]

    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })


train_entities
train_triplets # ground truth, remove interaction_type and choose random examples for few_shot
def transform_structure(data):
    output = {'sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

transformed = transform_structure(train_entities[0])
print(transformed)

def transform_structure(data):
    output = {'Sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

updated_train_entities = []
for i in range(len(train_entities)):
    transformed = transform_structure(train_entities[i])
    updated_train_entities.append(transformed)
print(updated_train_entities[-1])
len(updated_train_entities)

/tmp/ipython-input-888140995.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["type"] = train_df["type"].replace("No interaction", "no interaction")
/tmp/ipython-input-888140995.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["type"] = test_df["type"].replace("No interaction", "no interaction")
/tmp/ipython-input-888140995.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

{'sentence': '(1968, 1970), the higher serum concentrations of penicillins and cephaloridine reached after administration of probenecid are due not only to slower renal elimination but also to an altered distribution in the body. ', 'Entity_1': 'cephaloridine', 'Entity_2': 'penicillins', 'Entity_3': 'probenecid'}
{'Sentence': 'zaleplon did not affect the pharmacokinetics of ethanol.', 'Entity_1': 'ethanol', 'Entity_2': 'zaleplon'}


3706

In [ ]:
import numpy as np
import pandas as pd
import json

# Evaluation and Comparison

In [ ]:
with open('triplet_predictions_all_methods.json') as f:
    sentence_level_predictions = json.load(f)
sentence_level_predictions[3]

{'index': 3,
 'sentence': '(In some patients, the steroidal anti-inflammatory agent can reduce the diuretic, natriuretic, and antihypertensive effects of loop, potassium sparing, and thiazide diuretics.',
 'entities': ['loop diuretics',
  'potassium sparing diuretics',
  'steroidal anti-inflammatory agent',
  'thiazide diuretics'],
 'zero_shot_output': '(steroidal anti-inflammatory agent, int, loop diuretics)\n(steroidal anti-inflammatory agent, int, potassium sparing diuretics)\n(steroidal anti-inflammatory agent, int, thiazide diuretics)',
 'few_shot_output': '(steroidal anti-inflammatory agent, loop diuretics, effect)\n(steroidal anti-inflammatory agent, potassium sparing diuretics, effect)\n(steroidal anti-inflammatory agent, thiazide diuretics, effect)',
 'cot_output': '(steroidal anti-inflammatory agent, int, loop diuretics)\n(steroidal anti-inflammatory agent, int, potassium sparing diuretics)\n(steroidal anti-inflammatory agent, int, thiazide diuretics)'}

In [ ]:
with open('triplet_predictions_all_methods.json') as f:
    pair_level_predictions = json.load(f)
pair_level_predictions[3]

{'index': 3,
 'sentence': '(In some patients, the steroidal anti-inflammatory agent can reduce the diuretic, natriuretic, and antihypertensive effects of loop, potassium sparing, and thiazide diuretics.',
 'entities': ['loop diuretics',
  'potassium sparing diuretics',
  'steroidal anti-inflammatory agent',
  'thiazide diuretics'],
 'zero_shot_output': '(steroidal anti-inflammatory agent, int, loop diuretics)\n(steroidal anti-inflammatory agent, int, potassium sparing diuretics)\n(steroidal anti-inflammatory agent, int, thiazide diuretics)',
 'few_shot_output': '(steroidal anti-inflammatory agent, loop diuretics, effect)\n(steroidal anti-inflammatory agent, potassium sparing diuretics, effect)\n(steroidal anti-inflammatory agent, thiazide diuretics, effect)',
 'cot_output': '(steroidal anti-inflammatory agent, int, loop diuretics)\n(steroidal anti-inflammatory agent, int, potassium sparing diuretics)\n(steroidal anti-inflammatory agent, int, thiazide diuretics)'}

In [ ]:
len(pair_level_predictions)

3641

In [ ]:
triplet_sentences = {str(item["index"]): item["sentence"].strip() for item in sentence_level_predictions}
pair_sentences = {str(i): item["sentence"].strip() for i, item in enumerate(pair_level_predictions)}

# Find matches
matches = []
for tid, ts in triplet_sentences.items():
    for pid, ps in pair_sentences.items():
        if ts == ps:  # exact match
            matches.append((tid, pid, ts))

# Print results
for tid, pid, sent in matches:
    print(f"Match found:\n - Triplet index: {tid}\n - Pair index: {pid}\n - Sentence: {sent}\n")

Match found:
 - Triplet index: 0
 - Pair index: 0
 - Sentence: (1968, 1970), the higher serum concentrations of penicillins and cephaloridine reached after administration of probenecid are due not only to slower renal elimination but also to an altered distribution in the body.

Match found:
 - Triplet index: 1
 - Pair index: 1
 - Sentence: (Concurrent use with thiazide diuretics is not recommended, as they may provoke lithium toxicity because of reduced renal clearance.)

Match found:
 - Triplet index: 2
 - Pair index: 2
 - Sentence: (Concurrent use with thiazide diuretics may enhance the possibility of digitalis toxicity associated with hypokalemia.)

Match found:
 - Triplet index: 3
 - Pair index: 3
 - Sentence: (In some patients, the steroidal anti-inflammatory agent can reduce the diuretic, natriuretic, and antihypertensive effects of loop, potassium sparing, and thiazide diuretics.

Match found:
 - Triplet index: 4
 - Pair index: 4
 - Sentence: (Indomethacin) diuretics are used c

In [ ]:
import json
from sklearn.metrics import classification_report

# Load files
with open("triplet_predictions_all_methods.json") as f:
    triplet_preds = json.load(f)

with open("after_llm_update_df_ddi_predictions.json") as f:
    pair_preds = json.load(f)

# Create lookup by sentence text
triplet_lookup = {item["sentence"].strip(): item for item in triplet_preds}
pair_lookup = {item["sentence_text"].strip(): item for item in pair_preds}

# Collect comparison results
results = []
for sent, trip_item in triplet_lookup.items():
    if sent in pair_lookup:
        pair_item = pair_lookup[sent]
        results.append({
            "sentence": sent,
            "ground_truth": pair_item["ground_truth"],
            "pair_zero_shot": pair_item.get("zero_shot_pred"),
            "pair_zero_shot_desc": pair_item.get("zero_shot_desc_pred"),
            "pair_few_shot": pair_item.get("few_shot_pred"),
            "pair_cot": pair_item.get("cot_pred"),
        })

# Now evaluate each method
methods = ["pair_zero_shot", "pair_zero_shot_desc", "pair_few_shot", "pair_cot"]

for method in methods:
    y_true = [r["ground_truth"] for r in results]
    y_pred = [r[method] for r in results]
    print(f"=== Evaluation for {method} ===")
    print(classification_report(y_true, y_pred, zero_division=0))
    print()


=== Evaluation for pair_zero_shot ===
                precision    recall  f1-score   support

        advise       0.57      0.53      0.55       127
        effect       0.28      0.59      0.38       226
           int       0.01      0.25      0.01        12
       invalid       0.00      0.00      0.00         0
     mechanism       0.19      0.15      0.17       167
no interaction       0.99      0.06      0.12      1600

      accuracy                           0.15      2132
     macro avg       0.34      0.26      0.21      2132
  weighted avg       0.82      0.15      0.18      2132


=== Evaluation for pair_zero_shot_desc ===
                precision    recall  f1-score   support

        advise       0.53      0.46      0.49       127
        effect       0.37      0.65      0.47       226
           int       0.01      0.25      0.02        12
       invalid       0.00      0.00      0.00         0
     mechanism       0.25      0.56      0.34       167
no interaction    

In [ ]:
import json
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load files
with open("triplet_predictions_all_methods.json") as f:
    triplet_preds = json.load(f)

with open("after_llm_update_df_ddi_predictions.json") as f:
    pair_preds = json.load(f)

# Create lookup by sentence text
triplet_lookup = {item["sentence"].strip(): item for item in triplet_preds}
pair_lookup = {item["sentence_text"].strip(): item for item in pair_preds}

# Collect comparison results
results = []
for sent, trip_item in triplet_lookup.items():
    if sent in pair_lookup:
        pair_item = pair_lookup[sent]
        results.append({
            "sentence": sent,
            "ground_truth": pair_item["ground_truth"],
            "pair_zero_shot": pair_item.get("zero_shot_pred"),
            "pair_zero_shot_desc": pair_item.get("zero_shot_desc_pred"),
            "pair_few_shot": pair_item.get("few_shot_pred"),
            "pair_cot": pair_item.get("cot_pred"),
        })

# Methods to evaluate
METHOD_MAP = {
    "pair_zero_shot": "Zero-Shot",
    "pair_zero_shot_desc": "Zero-Shot + Desc",
    "pair_few_shot": "Few-Shot",
    "pair_cot": "CoT"
}
METHOD_ORDER = list(METHOD_MAP.keys())


def evaluate_sentence_level(results):
    """Compute global and per-class metrics for all methods"""
    global_metrics = []
    per_class_metrics = []

    y_true = [r["ground_truth"] for r in results]

    for method in METHOD_ORDER:
        if method not in results[0]:
            continue

        y_pred = [r[method] for r in results]
        model_name = METHOD_MAP[method]

        # Compute global metrics
        acc = accuracy_score(y_true, y_pred)
        micro_p, micro_r, micro_f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="micro", zero_division=0
        )
        macro_p, macro_r, macro_f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="macro", zero_division=0
        )
        weighted_p, weighted_r, weighted_f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="weighted", zero_division=0
        )

        global_metrics.append({
            "Model": model_name,
            "Accuracy": acc,
            "Micro_Precision": micro_p, "Micro_Recall": micro_r, "Micro_F1": micro_f1,
            "Macro_Precision": macro_p, "Macro_Recall": macro_r, "Macro_F1": macro_f1,
            "Weighted_Precision": weighted_p, "Weighted_Recall": weighted_r, "Weighted_F1": weighted_f1,
        })

        # Compute per-class metrics
        labels = sorted(set(y_true))
        precisions, recalls, f1s, supports = precision_recall_fscore_support(
            y_true, y_pred, labels=labels, zero_division=0
        )
        for lbl, p, r, f, s in zip(labels, precisions, recalls, f1s, supports):
            per_class_metrics.append({
                "Model": model_name,
                "Class": lbl,
                "Precision": p,
                "Recall": r,
                "F1": f,
                "Support": s
            })

    return pd.DataFrame(global_metrics), pd.DataFrame(per_class_metrics)


def generate_report(global_df, class_df):
    """Generate a comparison report in text format"""
    global_df = global_df.sort_values("Macro_F1", ascending=False)
    global_df = global_df.round(4)
    class_df = class_df.round(4)

    report = "Pair-LEVEL COMPARISON REPORT\n"
    report += "================================\n\n"
    report += "Global Metrics:\n"
    report += global_df.to_string(index=False) + "\n\n"
    report += "Per-Class Metrics:\n"
    report += class_df.to_string(index=False) + "\n"

    # Highlight best model
    best_model = global_df.iloc[0]
    report += (
        f"\nBest Model: {best_model['Model']} "
        f"(Macro F1={best_model['Macro_F1']:.4f}, Micro F1={best_model['Micro_F1']:.4f})"
    )

    return report


if __name__ == "__main__":
    global_results, class_results = evaluate_sentence_level(results)
    report = generate_report(global_results, class_results)
    print(report)

Pair-LEVEL COMPARISON REPORT

Global Metrics:
           Model  Accuracy  Micro_Precision  Micro_Recall  Micro_F1  Macro_Precision  Macro_Recall  Macro_F1  Weighted_Precision  Weighted_Recall  Weighted_F1
        Few-Shot    0.3218           0.3218        0.3218    0.3218           0.3469        0.3336    0.2871              0.8272           0.3218       0.4095
Zero-Shot + Desc    0.2092           0.2092        0.2092    0.2092           0.3580        0.3341    0.2492              0.8306           0.2092       0.2308
       Zero-Shot    0.1548           0.1548        0.1548    0.1548           0.3400        0.2639    0.2051              0.8220           0.1548       0.1755
             CoT    0.3124           0.3124        0.3124    0.3124           0.3198        0.1446    0.1668              0.6484           0.3124       0.4092

Per-Class Metrics:
           Model          Class  Precision  Recall     F1  Support
       Zero-Shot         advise     0.5726  0.5276 0.5492      127
     

In [ ]:
train_triplets_df = pd.DataFrame(train_triplets)
train_triplets_df
triplet_prediction_results = pd.DataFrame(sentence_level_predictions)
triplet_prediction_results
merged_df = train_triplets_df.merge(triplet_prediction_results)
# merged_df

In [ ]:
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Configuration
INTERACTION_TYPES = {"effect", "mechanism", "advise", "int", "no interaction"}
MODEL_MAP = {
    'zero_shot_output': 'Zero-Shot',
    'few_shot_output': 'Few-Shot',
    'cot_output': 'CoT'
}
MODEL_ORDER = list(MODEL_MAP.keys())

def parse_prediction_output(pred_output):
    """Robust parser that requires exactly 2 drugs and 1 interaction type"""
    if not isinstance(pred_output, str):
        return []

    # Find all triplets in the string
    triplet_pattern = r'\(([^,]+?),\s*([^,]+?),\s*([^)]+?)\)'
    raw_triplets = re.findall(triplet_pattern, pred_output)

    cleaned_triplets = []
    for triplet in raw_triplets:
        parts = [p.strip().lower() for p in triplet]

        # Identify interaction type candidates
        interaction_candidates = [p for p in parts if p in INTERACTION_TYPES]
        drug_candidates = [p for p in parts if p not in INTERACTION_TYPES]

        # Valid triplet: exactly 1 interaction + 2 drugs
        if len(interaction_candidates) == 1 and len(drug_candidates) == 2:
            itype = interaction_candidates[0]
            cleaned_triplets.append((drug_candidates[0], itype, drug_candidates[1]))
            continue

        # Fallback: positional checks (only if no clear match)
        if len(parts) == 3:
            # Case 1: Middle element is interaction
            if parts[1] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[0], parts[1], parts[2]))
            # Case 2: First element is interaction
            elif parts[0] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[1], parts[0], parts[2]))
            # Case 3: Last element is interaction
            elif parts[2] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[0], parts[2], parts[1]))

    return cleaned_triplets

def normalize_drug_pair(drug1, drug2):
    """Create canonical drug pair representation"""
    return tuple(sorted([drug1.strip().lower(), drug2.strip().lower()]))

def evaluate_predictions(df):
    """Comprehensive evaluation with model comparison metrics"""
    results = []
    per_class_results = []

    for model_col in MODEL_ORDER:
        if model_col not in df.columns:
            continue

        model_name = MODEL_MAP[model_col]

        # Initialize metrics
        metrics = {
            'Model': model_name,
            'Accuracy': 0,
            'Micro_Precision': 0, 'Micro_Recall': 0, 'Micro_F1': 0,
            'Macro_Precision': 0, 'Macro_Recall': 0, 'Macro_F1': 0,
            'Weighted_Precision': 0, 'Weighted_Recall': 0, 'Weighted_F1': 0,
        }

        class_metrics = {itype: {'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0} for itype in INTERACTION_TYPES}
        all_true = []
        all_pred = []

        for _, row in df.iterrows():
            # Process ground truth
            gold_triplets = set()
            for triplet in row['triplets']:
                if len(triplet) == 3:
                    d1, itype, d2 = [x.strip().lower() for x in triplet]
                    gold_triplets.add((normalize_drug_pair(d1, d2), itype))

            # Process predictions
            pred_triplets = set()
            for d1, itype, d2 in parse_prediction_output(row[model_col]):
                pred_triplets.add((normalize_drug_pair(d1, d2), itype))

            # Create lists for sklearn metrics
            for itype in INTERACTION_TYPES:
                # True positive: both gold and prediction have this interaction
                if any((dp, t) in gold_triplets and t == itype for dp, t in pred_triplets):
                    class_metrics[itype]['TP'] += 1
                    all_true.append(itype)
                    all_pred.append(itype)
                # False positive: prediction has this interaction but gold doesn't
                elif any(t == itype for dp, t in pred_triplets):
                    class_metrics[itype]['FP'] += 1
                    all_pred.append(itype)
                    # Find what the true label should be
                    true_label = "no interaction"  # Default if no interaction found
                    for dp, t in gold_triplets:
                        if dp in [dp_p for dp_p, t_p in pred_triplets if t_p == itype]:
                            true_label = t
                            break
                    all_true.append(true_label)
                # False negative: gold has this interaction but prediction doesn't
                elif any(t == itype for dp, t in gold_triplets):
                    class_metrics[itype]['FN'] += 1
                    all_true.append(itype)
                    all_pred.append("no interaction")  # Assuming prediction missed it

        # Calculate accuracy
        correct = sum(1 for t, p in zip(all_true, all_pred) if t == p)
        total = len(all_true) if all_true else 1
        metrics['Accuracy'] = correct / total if total > 0 else 0

        # Calculate micro, macro, and weighted metrics using sklearn
        if all_true and all_pred:
            micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='micro', zero_division=0)
            macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='macro', zero_division=0)
            weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='weighted', zero_division=0)

            metrics.update({
                'Micro_Precision': micro_precision,
                'Micro_Recall': micro_recall,
                'Micro_F1': micro_f1,
                'Macro_Precision': macro_precision,
                'Macro_Recall': macro_recall,
                'Macro_F1': macro_f1,
                'Weighted_Precision': weighted_precision,
                'Weighted_Recall': weighted_recall,
                'Weighted_F1': weighted_f1
            })


        results.append(metrics)

        # Calculate per-class metrics
        for itype in INTERACTION_TYPES:
            cm = class_metrics[itype]
            p = cm['TP'] / (cm['TP'] + cm['FP']) if cm['TP'] + cm['FP'] > 0 else 0
            r = cm['TP'] / (cm['TP'] + cm['FN']) if cm['TP'] + cm['FN'] > 0 else 0
            f = 2 * p * r / (p + r) if p + r > 0 else 0
            support = cm['TP'] + cm['FN']

            per_class_results.append({
                'Model': model_name,
                'Interaction': itype,
                'Precision': p,
                'Recall': r,
                'F1': f,
                'Support': support
            })

    return pd.DataFrame(results), pd.DataFrame(per_class_results)

def generate_comparison_report(global_df, class_df):
    """Create model comparison report with sorting"""
    # Sort models by Micro F1 score
    global_df = global_df.sort_values('Micro_F1', ascending=False)

    # Format numbers
    global_df = global_df.round(4)
    class_df = class_df.round(4)

    # Generate report
    report = "SENTENCE LEVEL MODEL COMPARISON REPORT\n"
    report += "=======================\n\n"
    report += "Global Metrics:\n"
    report += global_df.to_string(index=False) + "\n\n"
    report += "Per-Class Metrics:\n"
    report += class_df.to_string(index=False)

    # Add performance comparison
    best_model = global_df.iloc[0]
    report += f"\n\nBest Model: {best_model['Model']} (Macro F1={best_model['Macro_F1']:.4f} and Micro F1={best_model['Micro_F1']:.4f})"

    return report

# Main execution
if __name__ == "__main__":

    global_results, class_results = evaluate_predictions(merged_df)
    report = generate_comparison_report(global_results, class_results)
    print(report)

SENTENCE LEVEL MODEL COMPARISON REPORT

Global Metrics:
    Model  Accuracy  Micro_Precision  Micro_Recall  Micro_F1  Macro_Precision  Macro_Recall  Macro_F1  Weighted_Precision  Weighted_Recall  Weighted_F1
 Few-Shot    0.5276           0.5276        0.5276    0.5276           0.4047        0.5215    0.4023              0.6233           0.5276       0.5643
Zero-Shot    0.4543           0.4543        0.4543    0.4543           0.4127        0.4820    0.3630              0.6208           0.4543       0.5200
      CoT    0.4137           0.4137        0.4137    0.4137           0.3733        0.4472    0.3102              0.5597           0.4137       0.4705

Per-Class Metrics:
    Model    Interaction  Precision  Recall     F1  Support
Zero-Shot no interaction     0.9617  0.2079 0.3419     2900
Zero-Shot      mechanism     0.4411  0.5787 0.5006      686
Zero-Shot         effect     0.2245  0.4052 0.2889      733
Zero-Shot         advise     0.5987  0.7952 0.6831      454
Zero-Shot       